In [ ]:
! pip install cmake dlib face-recognition opencv-python numpy

In [ ]:
import cv2
import face_recognition as fr
import numpy as np
import os

## Load and Convert Images

In [ ]:
path = "Images_Test"
images = []
classNames = []
myList = os.listdir(path)
myList.sort()
encodings = []

In [ ]:
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])

In [ ]:
print(classNames)

## Find Encodings

In [ ]:
import tqdm

In [ ]:
def findEncodings(img,name):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    encode = fr.face_encodings(img)[0]
    encodings.append(encode)

In [ ]:
for i in tqdm (range (len(classNames)), desc="Finding Encodings..."):
    img, name = images[i], classNames[i]
    findEncodings(img,name)
print("Encoding Complete!")

## Facial Recognition

In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
while True:
    success, img = cap.read()
    imgS = cv2.resize(img, (0,0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    facesCurFrame = fr.face_locations(imgS)
    encodesCurFrame = fr.face_encodings(imgS,facesCurFrame)

    for encodeFace, faceLoc in zip(encodesCurFrame,facesCurFrame):
        matches = fr.compare_faces(encodings,encodeFace)
        faceDis = fr.face_distance(encodings,encodeFace)
        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            y1,x2,y2,x1 = faceLoc
            y1, x2, y2, x1 = y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(img, (x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)

    cv2.imshow('Webcam',img)
    cv2.waitKey(1)
